<a href="https://colab.research.google.com/github/bf777/MesoNet/blob/master/mesonet_demo_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MesoNet
Welcome to MesoNet, a toolbox for segmenting mesoscale calcium images! This notebook will take you through all the steps needed to process your own calcium imaging dataset using our models.

In [1]:
# Clone MesoNet repository
!git clone https://github.com/bf777/MesoNet.git

fatal: destination path 'MesoNet' already exists and is not an empty directory.


# Prepare inputs

In [2]:
!mkdir /content/mesonet_inputs/
!mkdir /content/mesonet_inputs/pipeline1_2/

mkdir: cannot create directory ‘/content/mesonet_inputs/’: File exists
mkdir: cannot create directory ‘/content/mesonet_inputs/pipeline1_2/’: File exists


In [3]:
# Utility to install mesonet package and associated requirements (will be replaced with pip later)
!pip install matplotlib==3.1.3

# Install DeepLabCut for pose estimation
!pip install deeplabcut

  Using cached h5py-3.1.0-cp37-cp37m-manylinux1_x86_64.whl (4.0 MB)
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mesonet 1.0.4.1 requires keras==2.3.1, but you have keras 2.4.3 which is incompatible.
mesonet 1.0.4.1 requires numpy~=1.17.3, but you have numpy 1.19.5 which is incompatible.


Because of how DeepLabCut operates, you now need to restart your runtime (under the Runtime menu).

In [4]:
# NOTE: Rerun this cell and the following two cells if you're getting an error when importing MesoNet
%cd MesoNet/

/content/MesoNet


In [5]:
# Install MesoNet
!python setup.py install

# Reinstall OpenCV to address compatibility issue
!pip install opencv-python==4.4.0.46

# Reinstall h5py
!pip install h5py==2.10.0

# pip install mesonet

running install
running bdist_egg
running egg_info
writing mesonet.egg-info/PKG-INFO
writing dependency_links to mesonet.egg-info/dependency_links.txt
writing requirements to mesonet.egg-info/requires.txt
writing top-level names to mesonet.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'mesonet.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/model.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/data.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/utils.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/gui_train.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/gui_test.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/lib/mesonet/dlc_predict.py -> build/bdist.linux-x86_64/egg/mesonet
copying build/l

In [6]:
# Use tensorflow 1.x (supported by DeepLabCut)
%tensorflow_version 1.x
import tensorflow
import os
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


We will now pull information from the OSF repository containing the DeepLabCut and U-Net models for our code.

In [15]:
!pip install osfclient

In [16]:
%cd /content/
!osf -p svztu fetch 6_Landmark_estimation_model/atlas-DongshengXiao-2020-08-03.zip mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip

/content
100% 643M/643M [00:07<00:00, 82.4Mbytes/s]


In [17]:
!unzip -q mesonet_inputs/atlas-DongshengXiao-2020-08-03.zip -d /content/mesonet_inputs

In [18]:
%cd /content/
!osf -p svztu fetch 7_U-Net_model/DongshengXiao_brain_bundary.hdf5 MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5

/content
100% 373M/373M [00:04<00:00, 87.5Mbytes/s]


In [19]:
%cd /content/
!osf -p svztu fetch 7_U-Net_model/DongshengXiao_unet_motif_based_functional_atlas.hdf5 MesoNet/mesonet/models/DongshengXiao_unet_motif_based_functional_atlas.hdf5

/content
100% 373M/373M [00:04<00:00, 81.9Mbytes/s]


In [20]:
%cd /content/
!osf -p svztu fetch 8_VoxelMorph_model/VoxelMorph_Motif_based_functional_map_model_transformed1000.h5 mesonet_inputs/voxelMorph_Motif_based_functional_map_model_transformed1000.h5

/content
100% 1.51M/1.51M [00:00<00:00, 143Mbytes/s]


# Five ways to use MesoNet

MesoNet can be used through five approaches:
1. **Atlas to brain**: Given a pre-trained DeepLabCut model that was trained to associate anatomical landmarks with corresponding points on atlases of brain regions, register an atlas of brain regions to the fixed brain imaging data using affine transformations. This approach is useful if your data has common anatomical landmarks and is the most robust to variations in image quality and orientation within your data.
2. **Brain to atlas**: Given a pre-trained DeepLabCut model that was trained to associate anatomical landmarks with corresponding points on atlases of brain regions, the brain imaging data to a fixed atlas of brain regions using affine transformations. This approach is useful if you would like to normalize your brain images to a common template based on anatomical landmarks.
3. **Atlas to brain + sensory maps**: Given a pre-trained DeepLabCut model that was trained to associate anatomical landmarks with corresponding points on atlases of brain regions as well as a set of folders containing functional brain activity for that animal that is consistent across animals, register an atlas of brain regions to the fixed brain imaging data using affine transformations. This approach is useful if you have consistent peaks of functional activity across animals that you would like to use in the alignment processes.
4. **Motif-based functional maps (MBFMs) + U-Net**: Given a pre-trained U-Net model that was trained to associate brain imaging data with atlases of brain regions, predict the locations of brain regions in the data without the use of landmarks. The brain imaging data should be motif-based functional maps (MBFMs) calculated using the associated MATLAB code (using seqNMF). This approach is useful if one wishes to mark functional regions based on more complex features of the data (e.g. a motif-based functional map) than landmarks.
5. **Motif-based functional maps (MBFMs) + Brain-to-atlas + VoxelMorph**: Given a pre-trained VoxelMorph model that was trained to compute a non-linear transformation between a template functional brain atlas and brain image data, predict the locations of brain regions in the data. In particular, this approach can register each input brain image to a user-defined template functional atlas. The brain imaging data should be motif-based functional maps (MBFMs) calculated using the associated MATLAB code (using seqNMF). This approach is useful if your images are consistently oriented and you want to compare the predicted locations of brain regions across different images.

We will now copy over some sample input images for each of these five pipelines from OSF to our inputs folder. If, instead, you would like to upload your own images create a folder inside `mesonet_inputs` and put your images inside that folder; then, skip the next cell.

In [26]:
#@title Run this cell to fetch sample data from OSF
%cd /content/
!osf -p fy6e3 clone /content/mesonet_inputs/example_data

# # Pipeline 1 + 2
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline1_2/0.png mesonet_inputs/pipeline1_2/0.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline1_2/1.png mesonet_inputs/pipeline1_2/1.png

# # Pipeline 3
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/1/tail.png mesonet_inputs/pipeline3_sensory/sensory_maps/1/tail.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/1/visual.png mesonet_inputs/pipeline3_sensory/sensory_maps/1/visual.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/1/whisker.png mesonet_inputs/pipeline3_sensory/sensory_maps/1/whisker.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/2/tail.png mesonet_inputs/pipeline3_sensory/sensory_maps/2/tail.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/2/visual.png mesonet_inputs/pipeline3_sensory/sensory_maps/2/visual.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/2/whisker.png mesonet_inputs/pipeline3_sensory/sensory_maps/2/whisker.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/3/tail.png mesonet_inputs/pipeline3_sensory/sensory_maps/3/tail.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/3/visual.png mesonet_inputs/pipeline3_sensory/sensory_maps/3/visual.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_maps/3/whisker.png mesonet_inputs/pipeline3_sensory/sensory_maps/3/whisker.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/1.png mesonet_inputs/pipeline3_sensory/sensory_raw/1.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/2.png mesonet_inputs/pipeline3_sensory/sensory_raw/2.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/3.png mesonet_inputs/pipeline3_sensory/sensory_raw/3.png

# # Pipeline 4
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net/0.png mesonet_inputs/pipeline4_MBFM-U-Net/0.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net/1.png mesonet_inputs/pipeline4_MBFM-U-Net/1.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net/2.png mesonet_inputs/pipeline4_MBFM-U-Net/2.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net/3.png mesonet_inputs/pipeline4_MBFM-U-Net/3.png

# # Pipeline 5
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline5_VoxelMorph/1.png mesonet_inputs/pipeline5_VoxelMorph/1.png
# !osf -p svztu fetch 0_Example_data/Automated_pipeline_sample_data/pipeline5_VoxelMorph/2.png mesonet_inputs/pipeline5_VoxelMorph/2.png

/content
0files [00:00, ?files/s]
100% 15.2k/15.2k [00:00<00:00, 51.4Mbytes/s]
1files [00:06,  6.07s/files]
100% 15.6k/15.6k [00:00<00:00, 52.5Mbytes/s]
2files [00:07,  4.61s/files]
100% 21.0k/21.0k [00:00<00:00, 46.1Mbytes/s]
3files [00:09,  3.91s/files]
100% 21.3k/21.3k [00:00<00:00, 50.7Mbytes/s]
4files [00:10,  3.11s/files]
100% 19.6k/19.6k [00:00<00:00, 49.7Mbytes/s]
5files [00:12,  2.54s/files]
100% 19.5k/19.5k [00:00<00:00, 56.1Mbytes/s]
6files [00:13,  2.14s/files]
100% 13.8k/13.8k [00:00<00:00, 49.0Mbytes/s]
7files [00:17,  2.71s/files]
100% 13.8k/13.8k [00:00<00:00, 46.8Mbytes/s]
8files [00:18,  2.28s/files]
100% 13.4k/13.4k [00:00<00:00, 41.7Mbytes/s]
9files [00:19,  1.96s/files]
100% 157k/157k [00:00<00:00, 122Mbytes/s]
10files [00:23,  2.40s/files]
100% 146k/146k [00:00<00:00, 127Mbytes/s]
11files [00:24,  2.07s/files]
100% 167k/167k [00:00<00:00, 114Mbytes/s]
12files [00:26,  1.91s/files]
100% 135k/135k [00:00<00:00, 101Mbytes/s]
13files [00:28,  1.99s/files]
100% 120k/12

Now, input the information about your input and output images, as well as the U-Net and DeepLabCut models that you would like to use. The default values will use the test data that we've included in the MesoNet git repository (in `MesoNet/mesonet/tests/test_input`). If you're using your own input data, replace `input_filename` below with the name of a folder in `mesonet_inputs` containing your input data.

In [8]:
#@title Input information for the model
input_file_name = 'pipeline1_2'  #@param {type: "string"}
input_file_sensory_raw_name = 'sensory_raw'  #@param {type: "string"}
input_file_sensory_maps_name = 'sensory_maps'  #@param {type: "string"}
input_file_MBFM_name = 'pipeline4_MBFM-U-Net'  #@param {type: "string"}
input_file_voxelmorph_name = 'pipeline5_VoxelMorph'  #@param {type: "string"}

output_file_atlas_brain_name = 'mesonet_outputs_atlas_brain'  #@param {type: "string"}
output_file_brain_atlas_name = 'mesonet_outputs_brain_atlas'  #@param {type: "string"}
output_file_sensory_name = 'mesonet_outputs_sensory'  #@param {type: "string"}
output_file_MBFM_U_Net_name = 'mesonet_outputs_MBFM_U_Net'  #@param {type: "string"}
output_file_voxelmorph_name = 'mesonet_outputs_voxelmorph'  #@param {type: "string"}

model_name = 'DongshengXiao_brain_bundary.hdf5' #@param {type: "string"}
u_net_only_model_name = 'DongshengXiao_unet_motif_based_functional_atlas.hdf5'
dlc_model_name = 'atlas-DongshengXiao-2020-08-03'  #@param {type: "string"}
voxelmorph_model_name = 'voxelMorph_Motif_based_functional_map_model_transformed1000.h5'  #@param {type: "string"}

In [22]:
# Set up filepaths based on your inputs
input_path_root = os.path.join('/content','mesonet_inputs', 'example_data')
input_file = os.path.join(input_path_root, 'osfstorage', 'Automated_pipeline_sample_data', input_file_name)
input_file_sensory_raw = os.path.join(input_path_root, 
                                      'osfstorage', 'Automated_pipeline_sample_data', 'pipeline3_sensory', input_file_sensory_raw_name)
input_file_sensory_maps = os.path.join(input_path_root, 
                                       'osfstorage', 'Automated_pipeline_sample_data', 'pipeline3_sensory', input_file_sensory_maps_name)
input_file_MBFM = os.path.join(input_path_root, 
                               'osfstorage', 'Automated_pipeline_sample_data', input_file_MBFM_name)
input_file_voxelmorph = os.path.join(input_path_root, 
                                     'osfstorage', 'Automated_pipeline_sample_data', input_file_voxelmorph_name)

output_file_atlas_brain = os.path.join('/content','mesonet_outputs', output_file_atlas_brain_name)
output_file_brain_atlas = os.path.join('/content','mesonet_outputs', output_file_brain_atlas_name)
output_file_sensory = os.path.join('/content','mesonet_outputs', output_file_sensory_name)
output_file_MBFM_U_Net = os.path.join('/content','mesonet_outputs', output_file_MBFM_U_Net_name)
output_file_voxelmorph = os.path.join('/content','mesonet_outputs', output_file_voxelmorph_name)

model = os.path.join('/content', 'MesoNet', 'mesonet', 'models', model_name)
u_net_only_model = os.path.join('/content', 'MesoNet', 'mesonet', 'models', u_net_only_model_name)
voxelmorph_model = os.path.join('/content','mesonet_inputs', voxelmorph_model_name)
dlc_config = os.path.join('/content','mesonet_inputs', dlc_model_name, 'config.yaml')

In [10]:
!mkdir '/content/mesonet_outputs'
!mkdir '/content/mesonet_outputs/mesonet_outputs_atlas_brain'
!mkdir '/content/mesonet_outputs/mesonet_outputs_brain_atlas'
!mkdir '/content/mesonet_outputs/mesonet_outputs_sensory'
!mkdir '/content/mesonet_outputs/mesonet_outputs_MBFM_U_Net'
!mkdir '/content/mesonet_outputs/mesonet_outputs_voxelmorph'

mkdir: cannot create directory ‘/content/mesonet_outputs’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_atlas_brain’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_brain_atlas’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_sensory’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_MBFM_U_Net’: File exists
mkdir: cannot create directory ‘/content/mesonet_outputs/mesonet_outputs_voxelmorph’: File exists


Now that we've told Colab where to find the input and output folders, let's define the configuration file!

# Configure MesoNet

NOTE: If you get the error `ModuleNotFoundError: No module named 'mesonet'`, rerun the cell near the top of the notebook that starts with `%cd MesoNet/`, as well as the following cell.

In [11]:
# Set this environment variable to help MesoNet find the git repo location
os.environ["MESONET_GIT"]='/content/MesoNet/mesonet/'

In [23]:
# We need to make sure that DeepLabCut doesn't run with a GUI (which isn't
# supported in Colab).
os.environ["DLClight"]="True"

# Import mesonet and define the configuration file for each pipeline
import mesonet
## 1. Atlas to brain
# Atlas-to-brain warp with U-Net and DeepLabCut
print('1. Atlas-to-brain warp with U-Net and DeepLabCut')
config_file_atlas_brain = mesonet.config_project(input_file, output_file_atlas_brain, 'test', 
                                                 atlas_to_brain_align=True, use_voxelmorph=False, 
                                                 use_unet=True, use_dlc=True, sensory_match=False, 
                                                 mat_save=False, olfactory_check=True,
                                                 config=dlc_config, model=model)

## 2. Brain to atlas
# Brain-to-atlas warp with DeepLabCut
print('2. Brain-to-atlas warp with DeepLabCut')
config_file_brain_atlas = mesonet.config_project(input_file, output_file_brain_atlas, 'test', 
                                                 atlas_to_brain_align=False, use_voxelmorph=False, 
                                                 use_unet=True, use_dlc=True, sensory_match=False, 
                                                 mat_save=False, olfactory_check=True, 
                                                 config=dlc_config, model=model)

## 3. Atlas to brain + sensory
# Atlas-to-brain warp with U-Net, DeepLabCut, and sensory maps
print('3. Atlas-to-brain warp with U-Net, DeepLabCut, and sensory maps')
config_file_sensory = mesonet.config_project(input_file_sensory_raw, output_file_sensory, 'test',
                                             atlas_to_brain_align=True, use_voxelmorph=False, 
                                             use_unet=True, use_dlc=True, sensory_match=True, 
                                             sensory_path=input_file_sensory_maps, mat_save=False, 
                                             config=dlc_config, model=model)

## 4. MBFM + U-Net
# Motif-based functional maps (MBFMs) with atlas directly applied using U-Net
print('4. Motif-based functional maps (MBFMs) with atlas directly applied using U-Net')
config_file_MBFM_U_Net = mesonet.config_project(input_file_MBFM, output_file_MBFM_U_Net, 'test', 
                                                atlas_to_brain_align=True, use_voxelmorph=False, 
                                                use_unet=True, use_dlc=False, sensory_match=False, 
                                                mat_save=False, mask_generate=False, 
                                                config=dlc_config, model=u_net_only_model)

## 5. VoxelMorph
# Local deformation warp with VoxelMorph and DeepLabCut
print('5. Local deformation warp with VoxelMorph and DeepLabCut')
config_file_voxelmorph = mesonet.config_project(input_file_voxelmorph, output_file_voxelmorph, 'test', 
                                                atlas_to_brain_align=False, use_voxelmorph=True, 
                                                use_unet=True, use_dlc=True, sensory_match=False, mat_save=False, 
                                                config=dlc_config, model=model, 
                                                align_once=True, olfactory_check=True, 
                                                voxelmorph_model=voxelmorph_model)

1. Atlas-to-brain warp with U-Net and DeepLabCut
/content/MesoNet/mesonet/
2. Brain-to-atlas warp with DeepLabCut
/content/MesoNet/mesonet/
3. Atlas-to-brain warp with U-Net, DeepLabCut, and sensory maps
/content/MesoNet/mesonet/
4. Motif-based functional maps (MBFMs) with atlas directly applied using U-Net
/content/MesoNet/mesonet/
5. Local deformation warp with VoxelMorph and DeepLabCut
/content/MesoNet/mesonet/


The config file (by default in each of the output folders) contains information about how MesoNet will run for each pipeline. We'll be using these config files as an input to the last two functions needed to run MesoNet. 

Now, we will run each of the five pipelines in turn:



# Run MesoNet

## Pipeline 1: Atlas-to-brain
Firstly, we will identify the outer edges of the cortex:

### Predict regions

In [13]:
%cd /content/
mesonet.predict_regions(config_file_atlas_brain)

/content
/content/MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline1_2
2/2 [==============================] - 11s 5s/step


Next, we will identify and use cortical landmarks to align the atlas to the brain:

### Predict landmarks

In [14]:
mesonet.predict_dlc(config_file_atlas_brain)

2
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline1_2/0.png
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline1_2/1.png
Using snapshot-1030000 for model /content/mesonet_inputs/atlas-DongshengXiao-2020-08-03/dlc-models/iteration-2/atlasAug3-trainset95shuffle1


  0%|          | 0/2 [00:00<?, ?it/s]

Analyzing all the videos in the directory...
Starting to analyze %  /content/mesonet_outputs/mesonet_outputs_atlas_brain/dlc_output/tmp_video.mp4
/content/mesonet_outputs/mesonet_outputs_atlas_brain/dlc_output  already exists!
Loading  /content/mesonet_outputs/mesonet_outputs_atlas_brain/dlc_output/tmp_video.mp4
Duration of video [s]:  0.07 , recorded with  30.0 fps!
Overall # of frames:  2  found with (before cropping) frame dimensions:  512 512
Starting to extract posture


10it [00:02,  4.76it/s]              


Saving results in /content/mesonet_outputs/mesonet_outputs_atlas_brain/dlc_output...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
/content/mesonet_outputs/mesonet_outputs_atlas_brain/dlc_output  already exists!
Starting to process video: /content/mesonet_outputs/mesonet_outputs_atlas_brain/dlc_output/tmp_video.mp4
Loading /content/mesonet_outputs/mesonet_outputs_atlas_brain/dlc_output/tmp_video.mp4 and data.
No filtered data file found in /content/mesonet_outputs/mesonet_outputs_atlas_brain/dlc_output for video tmp_video and scorer DLC_resnet50_atlasAug3shuffle1_1030000.
Landmark prediction complete!
Performing first transformation of atlas 0...
Performing second transformation of atlas 0...
Performing first t

Congratulations, you're all done with this first pipeline! You can now check the outputs in the `mesonet_output_atlas_brain` folder. The segmented brain data can be found in `mesonet_output_atlas_brain/output_overlay`.

The following four pipelines will follow a similar pattern:

## Pipeline 2: Brain-to-atlas
This time, we will directly identify and use cortical landmarks to align the brain to the atlas:

### Predict regions

In [15]:
%cd /content/
mesonet.predict_regions(config_file_brain_atlas)

/content
/content/MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline1_2
2/2 [==============================] - 11s 5s/step


### Predict landmarks

In [16]:
%cd /content/
mesonet.predict_dlc(config_file_brain_atlas)

/content
2
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline1_2/0.png
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline1_2/1.png
Using snapshot-1030000 for model /content/mesonet_inputs/atlas-DongshengXiao-2020-08-03/dlc-models/iteration-2/atlasAug3-trainset95shuffle1


  0%|          | 0/2 [00:00<?, ?it/s]

Analyzing all the videos in the directory...
Starting to analyze %  /content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output/tmp_video.mp4
/content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output  already exists!
Loading  /content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output/tmp_video.mp4
Duration of video [s]:  0.07 , recorded with  30.0 fps!
Overall # of frames:  2  found with (before cropping) frame dimensions:  512 512
Starting to extract posture


10it [00:02,  4.91it/s]              

Saving results in /content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


/content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output  already exists!
Starting to process video: /content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output/tmp_video.mp4
Loading /content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output/tmp_video.mp4 and data.
No filtered data file found in /content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output for video tmp_video and scorer DLC_resnet50_atlasAug3shuffle1_1030000.
Landmark prediction complete!
Performing first transformation of atlas 0...
Performing second transformation of atlas 0...
Performing first transformation of atlas 1...
Performing second transformation of atlas 1...
LEN CNTS: 44
LEN LABELS: 44
Mask 0 saved!
LEN CNTS: 44
LEN LABELS: 44
Mask 1 saved!
Analysis complete! Check the outputs in the folders of /content/mesonet_outputs/mesonet_outputs_brain_atlas/dlc_output/../output_overlay.


You can now check the outputs in the `mesonet_output_brain_atlas` folder. The segmented brain data can be found in `mesonet_output_brain_atlas/output_overlay`.

## Pipeline 3: Atlas-to-Brain + sensory
Now, we return to the Brain-to-Atlas approach while also using peaks of functional activity that are common across animals as a further alignment step.

### Predict regions

In [24]:
%cd /content/
mesonet.predict_regions(config_file_sensory)

/content
/content/MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw
3/3 [==============================] - 16s 5s/step


### Predict landmarks

In [25]:
mesonet.predict_dlc(config_file_sensory)

3
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/1.png
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/2.png
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline3_sensory/sensory_raw/3.png
Using snapshot-1030000 for model /content/mesonet_inputs/atlas-DongshengXiao-2020-08-03/dlc-models/iteration-2/atlasAug3-trainset95shuffle1


  0%|          | 0/3 [00:00<?, ?it/s]

Analyzing all the videos in the directory...
Starting to analyze %  /content/mesonet_outputs/mesonet_outputs_sensory/dlc_output/tmp_video.mp4
/content/mesonet_outputs/mesonet_outputs_sensory/dlc_output  already exists!
Loading  /content/mesonet_outputs/mesonet_outputs_sensory/dlc_output/tmp_video.mp4
Duration of video [s]:  0.1 , recorded with  30.0 fps!
Overall # of frames:  3  found with (before cropping) frame dimensions:  512 512
Starting to extract posture


10it [00:02,  3.57it/s]              

Saving results in /content/mesonet_outputs/mesonet_outputs_sensory/dlc_output...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


...

Landmark prediction complete!


Performing first transformation of atlas 0...
Performing second transformation of atlas 0...


Performing first transformation of atlas 1...
Performing second transformation of atlas 1...


Performing first transformation of atlas 2...
Performing second transformation of atlas 2...


LEN CNTS: 49
LEN LABELS: 49
Mask 0 saved!
LEN CNTS: 46
LEN LABELS: 46
Mask 1 saved!
LEN CNTS: 44
LEN LABELS: 44
Mask 2 saved!
Analysis complete! Check the outputs in the folders of /content/mesonet_outputs/mesonet_outputs_sensory/dlc_output/../output_overlay.


You can now check the outputs in the `mesonet_output_sensory` folder. The segmented brain data can be found in `mesonet_output_sensory/output_overlay`.

## Pipeline 4: MBFM + U-Net
Our input for this pipeline will be a set of motif-based functional maps (MBFMs) - brain images that summarize patterns of spatio-temporal activity that are common across animals. You can generate these using a MATLAB script running [seqNMF](https://github.com/FeeLab/seqNMF) - such a script is available in `4_Data_code/New_end_to_end_code` on our [OSF repository](https://osf.io/svztu/). We will then use a U-Net model to directly segment the brain image into functional regions - no need for atlas registration here!

### Predict regions

In [26]:
%cd /content/
mesonet.predict_regions(config_file_MBFM_U_Net)

/content
/content/MesoNet/mesonet/models/DongshengXiao_unet_motif_based_functional_atlas.hdf5
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline4_MBFM-U-Net
4/4 [==============================] - 20s 5s/step


LEN CNTS: 44
LEN LABELS: 44
Mask 0 saved!
LEN CNTS: 54
LEN LABELS: 54
Mask 1 saved!
LEN CNTS: 52
LEN LABELS: 52
Mask 2 saved!
LEN CNTS: 42
LEN LABELS: 42
Mask 3 saved!
Analysis complete! Check the outputs in the folders of /content/mesonet_outputs/mesonet_outputs_MBFM_U_Net/output_overlay.


You can now check the outputs in the `mesonet_output_MBFM_U_Net` folder. The segmented brain data can be found in `mesonet_output_MBFM_U_Net/output_overlay`.

## Pipeline 5: VoxelMorph
Our input for this pipeline will be a raw brain image followed by an MBFM (see Pipeline 4 description for details). We will use VoxelMorph - a local deformation technique - to register the MBFM to an atlas based on a template image in the MesoNet repository.

### Predict regions

In [20]:
%cd /content/
mesonet.predict_regions(config_file_voxelmorph)

/content
/content/MesoNet/mesonet/models/DongshengXiao_brain_bundary.hdf5
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline5_VoxelMorph
2/2 [==============================] - 10s 5s/step


### Predict landmarks

In [21]:
mesonet.predict_dlc(config_file_voxelmorph)

2
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline5_VoxelMorph/0.png
/content/mesonet_inputs/example_data/osfstorage/Automated_pipeline_sample_data/pipeline5_VoxelMorph/1.png
Using snapshot-1030000 for model /content/mesonet_inputs/atlas-DongshengXiao-2020-08-03/dlc-models/iteration-2/atlasAug3-trainset95shuffle1


  0%|          | 0/2 [00:00<?, ?it/s]

Analyzing all the videos in the directory...
Starting to analyze %  /content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output/tmp_video.mp4
/content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output  already exists!
Loading  /content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output/tmp_video.mp4
Duration of video [s]:  0.07 , recorded with  30.0 fps!
Overall # of frames:  2  found with (before cropping) frame dimensions:  512 512
Starting to extract posture


10it [00:02,  4.91it/s]              

Saving results in /content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output...
Saving csv poses!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


/content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output  already exists!
Starting to process video: /content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output/tmp_video.mp4
Loading /content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output/tmp_video.mp4 and data.
No filtered data file found in /content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output for video tmp_video and scorer DLC_resnet50_atlasAug3shuffle1_1030000.
Landmark prediction complete!
Performing first transformation of atlas 0...
Performing second transformation of atlas 0...
Performing first transformation of atlas 1...
Performing second transformation of atlas 1...


Results saved!


/content/MesoNet/mesonet/voxelmorph_align.py:101: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  x_data = rgb2gray(x_data)


LEN CNTS: 47
LEN LABELS: 47
Mask 0 saved!
LEN CNTS: 47
LEN LABELS: 47
Mask 1 saved!
Analysis complete! Check the outputs in the folders of /content/mesonet_outputs/mesonet_outputs_voxelmorph/dlc_output/../output_overlay.


You can now check the outputs in the `mesonet_output_voxelmorph` folder. The segmented brain data can be found in `mesonet_output_voxelmorph/output_overlay`.

# Conclusion
These five pipelines can be directly accessed in the graphical user interface (GUI) that is available for MesoNet for ease of use. Furthermore, you can customize your pipeline by changing the options defined in `mesonet.config_project` for the CLI and in the GUI - you can use our [Quick Start Guide](https://github.com/bf777/MesoNet/wiki/Quick-Start-Guide) and [Config File Guide](https://github.com/bf777/MesoNet/wiki/Config-File-Guide) for guidance.